In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
import pubchempy as pcp

In [26]:
compounds = pd.read_excel('DNA_Damage.xlsx')
compounds = compounds.iloc[:, 1:]

In [27]:
import pandas as pd
import pubchempy as pcp
from rdkit import Chem
from rdkit.Chem import Descriptors

def get_structural_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return {
        'MolWt': Descriptors.ExactMolWt(mol),
        'LogP': Descriptors.MolLogP(mol),
        'HBA': Descriptors.NumHAcceptors(mol),
        'HBD': Descriptors.NumHDonors(mol),
        'RotableBonds': Descriptors.NumRotatableBonds(mol),
        'TPSA': Descriptors.TPSA(mol)
    }

def get_pubchem_info(compound_name):
    try:
        results = pcp.get_compounds(compound_name, 'name')
        if results:
            return results[0].to_dict()
        return None
    except:
        return None

# 구조적 특성 분석
compounds['structural_features'] = compounds['CPD_SMILES'].apply(get_structural_features)

# PubChem 정보 수집
compounds['pubchem_info'] = compounds['CPD_NAME'].apply(get_pubchem_info)

# 결과 분석 및 개별 점수 저장
def analyze_compound(row):
    features = row['structural_features']
    pubchem = row['pubchem_info']
    
    features_score = 0
    bioactivity_score = 0
    
    # 구조적 특성 기반 점수
    if features['LogP'] > 5:  # 높은 지용성
        features_score += 1
    if features['MolWt'] < 500:  # 낮은 분자량
        features_score += 1
    
    # PubChem 생물학적 활성 기반 점수
    if pubchem and 'Bioactivity' in pubchem:
        if 'DNA damage' in pubchem['Bioactivity']:
            bioactivity_score += 2
    
    # 최종 합산 점수
    risk_score = features_score + bioactivity_score
    
    return pd.Series({
        'features_score': features_score,
        'bioactivity_score': bioactivity_score,
        'risk_score': risk_score
    })

# 개별 점수 추가 및 정렬
compounds[['features_score', 'bioactivity_score', 'risk_score']] = compounds.apply(analyze_compound, axis=1)
high_risk_compounds = compounds.sort_values('risk_score', ascending=False)

# 상위 10개의 결과 출력
print(high_risk_compounds[['CPD_NAME', 'features_score', 'bioactivity_score', 'risk_score']].head(10))

                    CPD_NAME  features_score  bioactivity_score  risk_score
1045    alpha-linolenic acid               2                  0           2
338               miconazole               2                  0           2
793               lorglumide               2                  0           2
615                verapamil               2                  0           2
795   oleanolic acid acetate               2                  0           2
312                 RU 28318               2                  0           2
125               salubrinal               2                  0           2
612                clofoctol               2                  0           2
611                  AY 9944               2                  0           2
797                 PK-11195               2                  0           2


In [35]:
compounds['pubchem_info'].iloc[1]

{'atom_stereo_count': 1,
 'atoms': [{'aid': 1, 'number': 17, 'element': 'Cl', 'x': 2, 'y': -4.75},
  {'aid': 2, 'number': 8, 'element': 'O', 'x': 5.4641, 'y': 1.25},
  {'aid': 3, 'number': 7, 'element': 'N', 'x': 3.732, 'y': 0.25},
  {'aid': 4, 'number': 7, 'element': 'N', 'x': 3.732, 'y': -1.75},
  {'aid': 5, 'number': 6, 'element': 'C', 'x': 3.732, 'y': 1.25},
  {'aid': 6, 'number': 6, 'element': 'C', 'x': 4.5981, 'y': -0.25},
  {'aid': 7, 'number': 6, 'element': 'C', 'x': 2.866, 'y': -0.25},
  {'aid': 8, 'number': 6, 'element': 'C', 'x': 4.5981, 'y': 1.75},
  {'aid': 9, 'number': 6, 'element': 'C', 'x': 4.5981, 'y': -1.25},
  {'aid': 10, 'number': 6, 'element': 'C', 'x': 2.866, 'y': -1.25},
  {'aid': 11, 'number': 6, 'element': 'C', 'x': 4.5981, 'y': 2.75},
  {'aid': 12, 'number': 6, 'element': 'C', 'x': 5.4641, 'y': 3.25},
  {'aid': 13, 'number': 6, 'element': 'C', 'x': 3.732, 'y': 3.25},
  {'aid': 14, 'number': 6, 'element': 'C', 'x': 3.732, 'y': -2.75},
  {'aid': 15, 'number': 6,

In [3]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdFMCS

def get_common_substructures(smiles_list, min_atoms=5):
    mols = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]
    mcs = rdFMCS.FindMCS(mols, threshold=0.8, completeRingsOnly=True, atomCompare=rdFMCS.AtomCompare.CompareAny, bondCompare=rdFMCS.BondCompare.CompareAny, matchValences=False)
    return Chem.MolToSmarts(Chem.MolFromSmarts(mcs.smartsString))

high_risk_compounds = compounds  # 이미 DNA damage 가능성이 높다고 예측된 리스트
common_substructure = get_common_substructures(high_risk_compounds['CPD_SMILES'])

print("Common substructure in high-risk compounds:")
print(common_substructure)

mol = Chem.MolFromSmarts(common_substructure)
img = Draw.MolToImage(mol)
img.save("common_substructure.png")

Common substructure in high-risk compounds:
[#6,#7,#8,#16,#17]-,=,#;!@[#7,#6,#8]-,=;!@[#6,#7,#15,#16](=,-;!@[#8,#6,#7,#16])-,=;!@[#8,#6,#7,#15,#16]-,=;!@[#7,#6,#8,#15,#16]=,-,#;!@[#6,#7,#8,#15,#16]-,=,#;!@[#6,#7,#8,#16,#17,#53]


In [11]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors

def analyze_structural_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return pd.Series({
        'MolWt': Descriptors.ExactMolWt(mol),
        'LogP': Descriptors.MolLogP(mol),
        'HBA': Descriptors.NumHAcceptors(mol),
        'HBD': Descriptors.NumHDonors(mol),
        'TPSA': Descriptors.TPSA(mol),
        'RotableBonds': Descriptors.NumRotatableBonds(mol),
        'AromaticRings': Descriptors.NumAromaticRings(mol),
    })

# 구조적 특성 분석
high_risk_features = high_risk_compounds['CPD_SMILES'].apply(analyze_structural_features)

high_risk_features['CPD_NAME'] = high_risk_compounds['CPD_NAME']

# 결과 요약
feature_summary = high_risk_features.drop('CPD_NAME', axis=1).agg(['mean', 'std', 'min', 'max'])
print("Structural features of high-risk compounds:")
print(feature_summary)

Structural features of high-risk compounds:
            MolWt      LogP        HBA        HBD        TPSA  RotableBonds  \
mean   344.995299  2.936706   4.456064   1.606391   70.678097      4.410312   
std    118.907321  1.758813   2.508357   1.478200   42.296623      3.460026   
min    105.078979 -2.734000   0.000000   0.000000    0.000000      0.000000   
max   1201.841368  9.465200  19.000000  12.000000  315.210000     26.000000   

      AromaticRings  
mean       1.722585  
std        1.068180  
min        0.000000  
max        8.000000  


In [7]:
import matplotlib.pyplot as plt
import seaborn as sns

# 박스 플롯 생성
plt.figure(figsize=(12, 6))
sns.boxplot(data=high_risk_features)
plt.title("Distribution of Structural Features in High-Risk Compounds")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("structural_features_boxplot.png")
plt.close()

# 상관관계 히트맵 생성
plt.figure(figsize=(10, 8))
sns.heatmap(high_risk_features.corr(), annot=True, cmap='coolwarm')
plt.title("Correlation of Structural Features in High-Risk Compounds")
plt.tight_layout()
plt.savefig("structural_features_correlation.png")
plt.close()

In [12]:
# 각 특성의 평균 계산
mean_tpsa = high_risk_features['TPSA'].mean()
mean_hba = high_risk_features['HBA'].mean()
mean_hbd = high_risk_features['HBD'].mean()

# TPSA, HBA, HBD가 모두 평균보다 높은 화합물 선별
high_all_features = high_risk_features[
    (high_risk_features['TPSA'] > mean_tpsa) &
    (high_risk_features['HBA'] > mean_hba) &
    (high_risk_features['HBD'] > mean_hbd)
]

print(f"Number of compounds with high TPSA, HBA, and HBD: {len(high_all_features)}")
print("\nSelected compounds:")
print(high_all_features)

Number of compounds with high TPSA, HBA, and HBD: 302

Selected compounds:
           MolWt    LogP   HBA  HBD    TPSA  RotableBonds  AromaticRings  \
0     382.983497  0.9365   6.0  3.0  118.36           5.0            1.0   
11    525.103513 -0.6532  13.0  3.0  172.46          10.0            2.0   
13    633.285716  5.6617   6.0  4.0  107.47           1.0            2.0   
15    398.137890  3.6758   5.0  2.0  103.12           4.0            4.0   
35    255.013619  1.5261   5.0  2.0   85.08           3.0            2.0   
...          ...     ...   ...  ...     ...           ...            ...   
1346  356.083078  3.4918   6.0  2.0   80.15           4.0            2.0   
1362  267.096754 -1.9800   9.0  4.0  139.54           2.0            2.0   
1363  267.096754 -1.9800   9.0  4.0  139.54           2.0            2.0   
1367  397.109627  3.5259   8.0  2.0   97.47           8.0            3.0   
1370  674.241035  4.7958   8.0  3.0  146.74           8.0            4.0   

           C

In [13]:
# 75번째 백분위수 계산
percentile_75_tpsa = high_risk_features['TPSA'].quantile(0.75)
percentile_75_hba = high_risk_features['HBA'].quantile(0.75)
percentile_75_hbd = high_risk_features['HBD'].quantile(0.75)

# TPSA, HBA, HBD가 모두 75번째 백분위수보다 높은 화합물 선별
high_all_features_strict = high_risk_features[
    (high_risk_features['TPSA'] > percentile_75_tpsa) &
    (high_risk_features['HBA'] > percentile_75_hba) &
    (high_risk_features['HBD'] > percentile_75_hbd)
]

print(f"Number of compounds with very high TPSA, HBA, and HBD: {len(high_all_features_strict)}")
print("\nSelected compounds (strict criteria):")
print(high_all_features_strict)

Number of compounds with very high TPSA, HBA, and HBD: 145

Selected compounds (strict criteria):
           MolWt     LogP   HBA  HBD    TPSA  RotableBonds  AromaticRings  \
0     382.983497  0.93650   6.0  3.0  118.36           5.0            1.0   
11    525.103513 -0.65320  13.0  3.0  172.46          10.0            2.0   
13    633.285716  5.66170   6.0  4.0  107.47           1.0            2.0   
43    685.462579  1.46720   8.0  8.0  223.26          22.0            0.0   
44    611.310769  2.71720   6.0  3.0  118.21           5.0            3.0   
...          ...      ...   ...  ...     ...           ...            ...   
1324  580.179206 -1.16520  14.0  8.0  225.06           6.0            2.0   
1332  332.089603  2.64064   6.0  3.0  113.29           4.0            2.0   
1362  267.096754 -1.98000   9.0  4.0  139.54           2.0            2.0   
1363  267.096754 -1.98000   9.0  4.0  139.54           2.0            2.0   
1370  674.241035  4.79580   8.0  3.0  146.74           

In [18]:
high_all_features_strict['CPD_NAME'].tolist()

['altizide',
 'cefotiam',
 'penitrem A',
 'pepstatin',
 'DIHYDROERGOCRISTINE',
 'cucurbitacin I',
 'lasalocid',
 'glipizide',
 'cefdinir',
 'IB-MECA',
 'lasalocid',
 'cefuroxime',
 'methotrexate',
 'esculin',
 'succinylsulfathiazole',
 'troxerutin',
 'FPL 55712',
 'UBP 296',
 'crustecdysone',
 'ancitabine',
 'vincristine',
 'bromocriptine',
 'bromocriptine',
 'ciclosporin',
 'quercitrin',
 'bromocriptine',
 'CV 1808',
 '2-Cl-IB-MECA',
 'dihydro-ergotamine',
 'neohesperidin dihydrochalcone',
 'trifluridine',
 "2'-MeCCPA",
 'cefprozil',
 'FR 139317',
 'HSCI1_000325',
 'CGS 21680',
 "5-fluoro-5'-deoxyuridine",
 'pravastatin',
 'N6-Cyclopentyladenosine',
 'aconitine',
 'cefadroxil',
 'metrizamide',
 'calyculin A',
 'convallatoxin',
 'NSC23766',
 'tyrphostin AG 538',
 'cianidanol',
 'cladribine',
 "5'-N-ethylcarboxamidoadenosine",
 'minocycline',
 'ouabain',
 'minoxidil',
 'metacycline',
 'L 755507',
 'clofarabine',
 'asiaticoside',
 'deferoxamine',
 'vinblastine sulfate',
 'A-23187',
 'fun

In [5]:
from rdkit.Chem import DataStructs
def get_morgan_fingerprint(smiles, radius=2, nBits=2048):
    mol = Chem.MolFromSmiles(smiles)
    return AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=nBits)

high_risk_fps = [get_morgan_fingerprint(smiles) for smiles in high_risk_compounds['CPD_SMILES']]
similarity_matrix = []
for i in range(len(high_risk_fps)):
    sim_row = [DataStructs.TanimotoSimilarity(high_risk_fps[i], high_risk_fps[j]) for j in range(len(high_risk_fps))]
    similarity_matrix.append(sim_row)

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 10))
sns.heatmap(similarity_matrix, cmap='YlOrRd')
plt.title("Structural Similarity of High-Risk Compounds")
plt.savefig("similarity_heatmap.png")
plt.close()

[14:04:00] DEPRECATION WARNING: please use MorganGenerator
[14:04:00] DEPRECATION WARNING: please use MorganGenerator
[14:04:00] DEPRECATION WARNING: please use MorganGenerator
[14:04:00] DEPRECATION WARNING: please use MorganGenerator
[14:04:00] DEPRECATION WARNING: please use MorganGenerator
[14:04:00] DEPRECATION WARNING: please use MorganGenerator
[14:04:00] DEPRECATION WARNING: please use MorganGenerator
[14:04:00] DEPRECATION WARNING: please use MorganGenerator
[14:04:00] DEPRECATION WARNING: please use MorganGenerator
[14:04:00] DEPRECATION WARNING: please use MorganGenerator
[14:04:00] DEPRECATION WARNING: please use MorganGenerator
[14:04:00] DEPRECATION WARNING: please use MorganGenerator
[14:04:00] DEPRECATION WARNING: please use MorganGenerator
[14:04:00] DEPRECATION WARNING: please use MorganGenerator
[14:04:00] DEPRECATION WARNING: please use MorganGenerator
[14:04:00] DEPRECATION WARNING: please use MorganGenerator
[14:04:00] DEPRECATION WARNING: please use MorganGenerat

In [11]:
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform
import numpy as np

# 유사성 매트릭스를 거리 매트릭스로 변환
distance_matrix = 1 - np.array(similarity_matrix)

# 계층적 클러스터링 수행
linkage_matrix = linkage(squareform(distance_matrix), method='ward')

# 덴드로그램 그리기
plt.figure(figsize=(10, 7))
dendrogram(linkage_matrix)
plt.title('Hierarchical Clustering of High-Risk Compounds')
plt.xlabel('Compound Index')
plt.ylabel('Distance')
plt.savefig('compound_clustering.png')
plt.close()

In [12]:
from sklearn.cluster import KMeans

# Convert fingerprints to numpy array
fp_array = np.array(similarity_matrix)

# Perform K-means clustering
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(fp_array)

high_risk_compounds['Cluster'] = clusters

for cluster in range(5):
    cluster_compounds = high_risk_compounds[high_risk_compounds['Cluster'] == cluster]
    print(f"Cluster {cluster}:")
    print(cluster_compounds['CPD_NAME'].tolist())
    print("\n")

Cluster 0:
['equilin', 'SB-228357', 'KF 38789', 'bergaptene', 'papaverine hydrochloride', 'noscapine', 'diltiazem', 'esculin', 'salsolidine', 'gatifloxacin', 'colchicine', '40 (DMCM)', 'brazilin', 'troxerutin', 'L-655,708', 'L-701,252', 'vincristine', 'monastrol', 'quercitrin', 'sulpiride', "resveratrol 4'-methyl ether", 'totarol-19-carboxylic acid methyl ester', 'hydroquinine', 'neohesperidin dihydrochalcone', 'deoxysappanone B trimethyl ether', 'dihydroptaeroxylin', 'canadine', 'amisulpride', '7-Hydroxy-PIPAT maleate', 'L-stepholidine', 'veratric acid', 'A 61603 hydrobromide', '7-hydroxy-3-(2-methoxyphenyl)chromen-4-one', 'cromakalim', '5,7-dimethoxyflavanone', 'metaxalone', 'Ro 04-5595', 'aconitine', 'salsoline', 'ciprofibrate', 'maackiain', 'boldine', 'ungerine', 'tetrahydrotrimethylhispidin', 'estrone benzoate', 'gallopamil', 'Chromanol 293B', 'venlafaxine', 'cianidanol', 'naringenin', 'catechin pentabenzoate', 'SYM 2206', 'dehydrorotenone', 'cycloheximide-N-ethylethanoate', 'zaco